In [13]:
from tqdm import tqdm

import os
from glob import glob
import shutil
import json
from osgeo import gdal, osr
import numpy as np
import matplotlib.pyplot as plt
import re
from pyproj import CRS, Transformer
from PIL import Image
import requests

### 1. Reorder the dataset files

In [1]:
if not os.path.exists(f'./sorted'):
    os.mkdir(f'./sorted')

In [2]:
test_tif_img = []

path = "./data/test/flair_1_toy_aerial_test"

for file_name in glob(path+'/*/*/img/*.tif'):
    test_tif_img.append(file_name)

In [3]:
def convert_path_img_to_msk(path):
    return (
        path
        .replace('aerial', 'labels')
        .replace('img', 'msk')
        .replace('IMG', 'MSK')
    )

test_tif_msk = [convert_path_img_to_msk(p) for p in test_tif_img]

In [4]:
data_path_zip = list(zip(test_tif_msk, test_tif_img))
for (mask, img) in data_path_zip:
    for path in [mask, img]:
        basename = os.path.basename(path)
        folder = basename.split('_')[1].split('.')[0]

        if not os.path.exists(f'./sorted/{folder}'):
            os.mkdir(f'./sorted/{folder}')

        shutil.copyfile(path, f'./sorted/{folder}/{basename}')

### 2. Function definitions

In [5]:
LUT = [
    {"color": "#db0e9a", "class": "building"},
    {"color": "#938e7b", "class": "pervious surface"},
    {"color": "#f80c00", "class": "impervious surface"},
    {"color": "#a97101", "class": "bare soil"},
    {"color": "#1553ae", "class": "water"},
    {"color": "#194a26", "class": "coniferous"},
    {"color": "#46e483", "class": "deciduous"},
    {"color": "#f3a60d", "class": "brushwood"},
    {"color": "#660082", "class": "vineyard"},
    {"color": "#55ff00", "class": "herbaceous vegetation"},
    {"color": "#fff30d", "class": "agricultural land"},
    {"color": "#e4df7c", "class": "plowed land"},
    {"color": "#3de6eb", "class": "swimming pool"},
    {"color": "#ffffff", "class": "snow"},
    {"color": "#8ab3a0", "class": "clear cut"},
    {"color": "#6b714f", "class": "mixed"},
    {"color": "#c5dc42", "class": "ligneous"},
    {"color": "#9999ff", "class": "greenhouse"},
    {"color": "#000000", "class": "other"}
]

In [7]:
def convert_to_seg(image_array, lut):
    def hex_to_tuple(hex):
        h = hex.lstrip('#')
        return tuple(int(h[i:i+2], 16) for i in (0, 2, 4))

    for i in range(image_array.shape[0]):
        for j in range(image_array.shape[1]):
            class_id = image_array[i, j][0]
            
            color = hex_to_tuple(lut[class_id-1]["color"]) if class_id < len(lut) else (0,0,0)

            image_array[i, j] = color

    # Convert the NumPy array back to an image and save it to a file
    return image_array

In [8]:
def get_osm_data(latlon):
    lat, lon = latlon
    url = f'https://nominatim.openstreetmap.org/reverse?format=json&lat={lat}&lon={lon}&zoom=18&addressdetails=1'
    response = requests.get(url)
    return response.json()


def convert_to_latlon(coord, proj):
    if not proj:
        proj = 2154

    input_crs = CRS(f"EPSG:{proj}")
    wgs84_crs = CRS("EPSG:4326")  # WGS 84 (latitude et longitude)
    transformer = Transformer.from_crs(input_crs, wgs84_crs)
    return transformer.transform(*coord)


def tif_to_jpg(input_path, output_path, LUT):
    input_tif = gdal.Open(input_path)
    rgb_data = np.zeros((input_tif.RasterYSize, input_tif.RasterXSize, 3), dtype=np.uint8)

    if(input_tif.RasterCount > 3):
        for i in range(3):
            rgb_data[..., i] = input_tif.GetRasterBand(i+1).ReadAsArray()
    else:
        rgb_data[..., 0] = input_tif.GetRasterBand(1).ReadAsArray()
        rgb_data = convert_to_seg(rgb_data, LUT)

    # Close input TIFF file
    input_tif = None

    img = Image.fromarray(rgb_data)
    img.save(output_path)


def get_tif_metadata(input_path):
    tif_dataset = gdal.Open(input_path)

    # Get the origin, dimensions, and CRS
    geotransform = tif_dataset.GetGeoTransform()
    name = os.path.basename(input_path)
    origin = (geotransform[0], geotransform[3])
    dimensions = (tif_dataset.RasterXSize, tif_dataset.RasterYSize)

    # Get the CRS
    crs_wkt = tif_dataset.GetProjection()
    crs = osr.SpatialReference()
    crs.ImportFromWkt(crs_wkt)

    # Get the units
    unit_type = crs.GetLinearUnitsName()

    srs = osr.SpatialReference()
    srs.ImportFromWkt(tif_dataset.GetProjection())
    code = srs.GetAuthorityCode(None)

    latlon = convert_to_latlon(origin, code)

    osm_data = get_osm_data(latlon)

    # Create a dictionary to store the metadata
    metadata = {
        "origin": origin,
        "dimensions": dimensions,
        "unit_system": unit_type,
        "code": code,
        "latlong": latlon,
        **osm_data        
    } 

    return metadata

### 3. Convert images and query metadata

In [9]:
tifs = glob('./sorted/*/*.tif')

In [11]:
# convert image
for tif_path in tqdm(tifs):
    dirname = os.path.dirname(tif_path)
    basename = os.path.basename(tif_path).split('.')[0]
    jpg_path = f"{dirname}/{basename}.png"
    tif_to_jpg(tif_path, jpg_path, LUT)

100%|██████████| 100/100 [01:36<00:00,  1.03it/s]


In [12]:
# get metadata
for tif_path in tqdm(tifs):
    dirname = os.path.dirname(tif_path)
    basename = os.path.basename(tif_path).split('.')[0]

    imgcode = basename.split('_')[1]
    json_path = f"{dirname}/{imgcode}.json"
    metadata = get_tif_metadata(tif_path)

    with open(json_path, "w") as fp:
        json.dump(metadata, fp)

100%|██████████| 100/100 [00:51<00:00,  1.95it/s]
